In [1]:
import pandas as pd
import numpy as np
import datetime
import requests
from bs4 import BeautifulSoup

### Web scraping for rain forecast

In [11]:
url_station = ["https://www.timeanddate.com/weather/@5805879/ext","https://www.timeanddate.com/weather/@5807487/ext","https://www.timeanddate.com/weather/@7289169/ext"  ]#urls for three locations
Data = []#empty list to append data
Station_info = []#empty list to append weather stations

for i in url_station:
    file = requests.get(i)#use request library to go webpage 
    soup = BeautifulSoup(file.content, "lxml")# use beaufifulSoup to get data from xml page
    table = soup.find('table', attrs = {'id': 'wt-ext'})#extract table from the webpage
    station = soup.title.text# return the title for each web page
    Station_info.append(station)# append web info
    
    for tr in table.find('tbody').find_all('tr'):#find 'tr' in the table body(each 'tr' represents the tablerow)
        dict = {}#empty dictionary
        dict['Date'] = tr.find('th').text#extract date i.e. table header
        all_td = tr.find_all('td')#find all columns ie table data/cell
        dict['PrecProb'] = all_td[7].text#extract text in 7th column which is probability of precipitation and add to dict
        dict['PrecAmount'] = all_td[8].text#extract amount of precipitation
        Data.append(dict)#append
        
print(Station_info)
#print(Data)

['Othello, Washington, USA 14 day weather forecast', 'Prosser, Washington, USA 14 day weather forecast', 'Tri-Cities, Washington, USA 14 day weather forecast']


In [12]:
Station= ['Othello', 'Prosser', 'TriCities']
df_forecast = pd.DataFrame(Data)
df_forecast['Station'] = np.repeat(Station, repeats = (df_forecast.shape[0])/(np.array(Station).shape[0]))
print(df_forecast.shape)
df_forecast.head()

(45, 4)


,Date,PrecProb,PrecAmount,Station
0,FriApr 9,1%,-,Othello
1,SatApr 10,2%,-,Othello
2,SunApr 11,1%,-,Othello
3,MonApr 12,46%,"0.02""",Othello
4,TueApr 13,5%,-,Othello


In [13]:
df_forecast['Date'] = df_forecast['Date'].str[3:]#remove first three characters i.e. days info
df_forecast1 = df_forecast.join(df_forecast['Date'].str.split(' ', 1, expand = True).rename(columns = {0:'Month', 1:'Day'}))#split Date column to month and Day
df_forecast1['PrecProb'] = (df_forecast1['PrecProb'].str.replace('%', '')).astype(int)#remove % and change to integer
df_forecast1['PrecAmount'] = df_forecast1['PrecAmount'].str.replace('"', '').replace('-', '0.00').astype(float)#remove " and - form PrecAmount column
df_forecast1.head()

,Date,PrecProb,PrecAmount,Station,Month,Day
0,Apr 9,1,0.00,Othello,Apr,9
1,Apr 10,2,0.00,Othello,Apr,10
2,Apr 11,1,0.00,Othello,Apr,11
3,Apr 12,46,0.02,Othello,Apr,12
4,Apr 13,5,0.00,Othello,Apr,13


In [15]:
forecast_grouped = df_forecast1[(df_forecast1['PrecAmount'] >0.04) & (df_forecast1['PrecProb'] >30)].groupby(['Station', 'Month']).size().reset_index(name = 'RainyDays')
forecast_grouped.head()

,Station,Month,RainyDays


In [ ]:
#("https://www.timeanddate.com/weather/@5805879/ext")# Othello
# (https://www.timeanddate.com/weather/@5807487/ext)# Prosser
# (https://www.timeanddate.com/weather/@7289169/ext)# Tricities